## Fake News Detection
* Name:Kanza Rasheed
* Enrollment No: 1195567
* MS Data Science-2018
* Univerisy of Padova

#   Word2Vec

Word2Vec is an neural network model that maximizes similarity between contextual neighbors while minimizing similarity for unseen contexts. Word2vec takes as its input a large corpus of text and produces a vector space, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space.
Target Vector sizes are set at the beginning of the training process, so the vectors are dense and do not need dimensionality reduction techniques.



## 1.0 Description

## 1.1 Continuous Bag of Words


Training objective is to maiximize the probability of observing the correct target word $w_t$ given context words $w_{c1}, w_{c2}, ... w_{cj}$

 $$ C = -log p(w_t | w_{c1} ... w_{cj}) $$

<img src="Images/cbow.png ">

The prediction vector is set as an average of all the context word vectors

## 1.2 Skip-gram

Training objective is to maiximize the probability of observing the correct context words $w_{ci}$ given target word $w_{t}$

 $$ C = -\sum^{j}_{i=1}log p(w_{ci} | w_{t}) $$
 
In this case, the prediction vector is the the target word vector.

<img src="Images/skip-gram.png ">

## 1.3 Word2Vec Parameters
There are many parameters on this constructor; a few noteworthy arguments are:
- `size`: (default 100) The number of dimensions of the embedding, e.g. the length of the dense vector to represent each token (word)
- `window`: (default 5) Number of context words to observe in each direction
- `min_count`: (default 5) The minimum count of words to consider when training the model; words with an occurrence less than this count will be ignored.
- `sg` (Skip-Gram): '0' indicates CBOW model; '1' indicates Skip-Gram (default 0 or CBOW)
- `alpha`: Learning rate (initial); prevents model from over-correcting, enables finer tuning
- `iterations`: Number of passes through dataset
- `batch_words`: Number of words to sample from data during each pass
- `batch_words`: (default 3) The number of threads to use while training.






The word vectors are directions in space and can encode relationships between words.  

The proximity of words to each other can be calculated through their cosine similarity.

![](http://multithreaded.stitchfix.com/assets/images/blog/vectors.gif)

## 2.0 Implementation

In the next few steps,word2vec model is being implemented on both of the datasets.
The following steps are followed:
* Preprocessing data
* Tokenlization and punctutation cleaning
* Hypertuning Word2Vec model
* Choose the classifier
* Hypertuning classifier

In [1]:
# Mount volume of your Google Drive Folder
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [2]:
# Note: adding datasets folder to 'My Drive' is required!
# This defines file paths
ROOT = './datasets'
FAKE_OR_REAL = ROOT + '/fake_or_real_news/fake_or_real_news.csv'
LIAR_LIAR_TRAIN = ROOT + '/liar/train.tsv'
LIAR_LIAR_TEST = ROOT + '/liar/test.tsv'

In [3]:
# Import libraries
import nltk
import pandas as pd
import numpy as np
import math
import sklearn
import matplotlib
import matplotlib.pyplot as plt

## 2.1 'Liar Liar' dataset

## 2.1.1 Pre-Processing

In [5]:
# Import 'Liar Liar' dataset
# Note: the dataset has already been splitted in train and test dataset
liar_ds_train = pd.read_csv(LIAR_LIAR_TRAIN, delimiter='\t', header=None)
liar_ds_test = pd.read_csv(LIAR_LIAR_TEST, delimiter='\t', header=None)

In [6]:
# Retrieve text (X) and label (Y) columns
# Only 'true' and 'false' values
# Only text (index=1) and labels (index=2) are mantained

# Train dataset
liar_ds_train = liar_ds_train.loc[liar_ds_train[1].isin(['true', 'false']), 1:2]
# Test dataset
liar_ds_test = liar_ds_test.loc[liar_ds_test[1].isin(['true', 'false']), 1:2]

In [7]:
# Check train dataset
liar_ds_train.head()

,1,2
0,false,Says the Annies List political group supports ...
3,false,Health care reform legislation is likely to ma...
5,true,The Chicago Bears have had more starting quart...
12,false,When Mitt Romney was governor of Massachusetts...
16,true,McCain opposed a requirement that the governme...


In [9]:
# Define X (input text ) and Y (label) variables for each dataset

# Train dataset
X_train_liar = liar_ds_train[2]
Y_train_liar = liar_ds_train[1]
# Test dataset
X_test_liar = liar_ds_test[2]
Y_test_liar = liar_ds_test[1]

In [10]:
# Describe training dataset
X_train_liar.describe()

count                                                  3671
unique                                                 3661
top       On changing the rules for filibusters on presi...
freq                                                      3
Name: 2, dtype: object

In [11]:
# Describe testing dataset
X_test_liar.describe()

count                                                   457
unique                                                  457
top       In this judicial race, special interest groups...
freq                                                      1
Name: 2, dtype: object

In [12]:
# We noted that there are some repeated texts inside the train dataset (not unique ones)
# hence we drop those rows

# Find duplicated rows
duplicated = X_train_liar.duplicated()
duplicated = duplicated[duplicated == True].index

# Drop duplicated rows from X and Y
X_train_liar.drop(duplicated, inplace = True)
Y_train_liar.drop(duplicated, inplace = True)

# Describe training dataset
X_train_liar.describe()

count                                                  3661
unique                                                 3661
top       Ronald Reagans signature on the 1986 amnesty a...
freq                                                      1
Name: 2, dtype: object

In [13]:
# Labels values must be turned into 0|1 form 'false'|'true'
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(['false', 'true'])
# Encode labels
Y_train_liar = le.transform(Y_train_liar)
Y_test_liar = le.transform(Y_test_liar)

The steps of importing dataset, selecting required columns, removing duplicates and label encoding have been done by group member Damiano

In [14]:
#Liar Dataset
X_train, Y_train = X_train_liar.copy(), Y_train_liar.copy()
X_test,Y_test = X_test_liar.copy(), Y_test_liar.copy()

In [15]:
#tokenization of training set
cleaned_train = [nltk.word_tokenize(x) for x in X_train]
print(cleaned_train[0])

['Says', 'the', 'Annies', 'List', 'political', 'group', 'supports', 'third-trimester', 'abortions', 'on', 'demand', '.']


In [16]:
#tokenization of testing set
cleaned_test = [nltk.word_tokenize(x) for x in X_test]
print(cleaned_test[0])

['Building', 'a', 'wall', 'on', 'the', 'U.S.-Mexico', 'border', 'will', 'take', 'literally', 'years', '.']


In [17]:
# Word2Vec expects a list of lists 

def clean_text(tokenized_list):
    import string
    sw = nltk.corpus.stopwords.words('english')
    sw.append("“")
    sw.append("”")
    sw.append("’")
    sw.append("‘")
    sw.append("—")
    new_list = [[token.lower() for token in tlist if token not in string.punctuation 
                 and token.lower() not in sw] for tlist in tokenized_list]
    return new_list


In [18]:
#Creats list of lists
cleaned_train= clean_text(cleaned_train)
cleaned_test= clean_text(cleaned_test)

## 2.1.2 Word2Vec Model Training 

In [19]:
#importing gensim Word2Vec model
from gensim.models import Word2Vec
from sklearn.metrics import *


C:\Users\Hp\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


* Skip Gram Vs CBOW:

Skip-gram: works well with small amount of the training data, represents well even rare words or phrases.

CBOW: several times faster to train than the skip-gram, slightly better accuracy for the frequent words

In the Word2Vec model, default CBOW has been used

In [20]:
# The Word2Vec Model with some of the default parameters for training set
seed = 2
feature_size=100
word_model_train = Word2Vec(cleaned_train, seed = seed, min_count = 1, size = 100, 
                      window = 5, alpha = 0.025, iter = 10, 
                      batch_words = 10000, negative = 5)




We got ourselves a dictionary mapping word -> 100-dimensional vector. Now we can use it to build features. The simplest way to do that is by averaging word vectors for all words in a text. 

## 2.1.3 Word Vectorization

In [27]:
#averaging the vectors for training set
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords>0:
        feature_vector = np.divide(feature_vector, nwords)
    
    
    return feature_vector
    

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
w2v_feature_array_train = averaged_word_vectorizer(corpus=cleaned_train, model=word_model_train,
                                             num_features=100)




C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [26]:
#averaging the vectors for testing set

w2v_feature_array_test = averaged_word_vectorizer(corpus=cleaned_test, model=word_model_train,
                                             num_features=100)



C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


For testing purpose, I am using logistic regression as it is one of the best classifiers that gives good results with word embeddings. (This result was derived after testing embeddings with SVM, Random Forest and Naive Bayes).
In the later steps, Log regression and XGBoost classifiers will be tuned.

In [28]:
from sklearn.linear_model import LogisticRegression
LR0_model = LogisticRegression().fit(w2v_feature_array_train, Y_train)
y0_pred = LR0_model.predict(w2v_feature_array_test)
accuracy_score(Y_test, y0_pred)

0.5492341356673961

The accuracy with Liar dataset and word2Vec embeddings is only 54.9. In the next section, the same steps have been done for the fake or real dataset.

## 2.2 Fake or Real Dataset

## 2.2.1 Pre-Processing

In [29]:
# Import 'Fake or Real' dataset
fake_or_real = pd.read_csv(FAKE_OR_REAL, delimiter=',', header=0)

In [30]:
#
fake_or_real.drop('Unnamed: 0', inplace=True, axis=1)
# Find duplicate indexes
duplicated = fake_or_real.duplicated()
duplicated = duplicated[duplicated == True].index

# Remove duplicates
fake_or_real.drop(duplicated, inplace=True)

#changing the labels from fake and real to 0 and 1
label_trans = lambda i: 0 if i == 'FAKE' else 1
fake_or_real.label = fake_or_real.label.apply(label_trans)


The steps of importing data and removing duplicates have been done by group member Damiano

In [31]:
fake_or_real.label.head(20)

0     0
1     0
2     1
3     0
4     1
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    0
13    0
14    1
15    1
16    0
17    0
18    1
19    1
Name: label, dtype: int64

In [32]:
# Define X (input text) and Y (label) variables
X_fake_or_real = fake_or_real['text']+fake_or_real['title']
Y_fake_or_real = fake_or_real['label']

In [33]:
cleaned_X_fake_or_real = [nltk.word_tokenize(x) for x in X_fake_or_real]


In [34]:
# Clean the tokenized text 

def clean_text(tokenized_list):
    import string
    sw = nltk.corpus.stopwords.words('english')
    sw.append("“")
    sw.append("”")
    sw.append("’")
    sw.append("‘")
    sw.append("—")
    new_list = [[token.lower() for token in tlist if token not in string.punctuation 
                 and token.lower() not in sw] for tlist in tokenized_list]
    return new_list
cleaned_X_fake_or_real= clean_text(cleaned_X_fake_or_real)

## 2.2.2 Training of Word2Vec Model

In [35]:
from gensim.models import Word2Vec
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [36]:
# Word2Vec Model for cleaned fake or real dataset


seed = 2
feature_size=100
word_model_X_fake_or_real = Word2Vec(cleaned_X_fake_or_real, seed = seed, min_count = 1, size = 100, 
                                     window = 5, alpha = 0.025, iter = 10, 
                                     batch_words = 10000, negative = 5)






## 2.2.3 Word Vectorization

In [37]:
#averaging the vectors

def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords>0:
        feature_vector = np.divide(feature_vector, nwords)
    
    
    return feature_vector
    

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# get document level embeddings
w2v_feature_array = averaged_word_vectorizer(corpus=cleaned_X_fake_or_real, model=word_model_X_fake_or_real,
                                             num_features=100)



C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [38]:

x0_train, x0_test, y0_train, y0_test = train_test_split( w2v_feature_array, Y_fake_or_real, test_size=0.33, random_state=seed)
LR0_model = LogisticRegression().fit(x0_train, y0_train)
y0_pred = LR0_model.predict(x0_test)
accuracy_score(y0_test, y0_pred)

0.8981348637015782

## 2.3 Tuning of Word2Vec Model
Since accuracy on fake and real news dataset is much higher than the liar dataset,in the next steps I will try tuning the word2Vec model by changing different parameters.


## 2.3.1 Changing the size

In [39]:
test_size=0.33

In [40]:
# Testing the influence of size and use 50 as difference

for j in range(1,8):
    word_model_X_fake_or_real = Word2Vec(cleaned_X_fake_or_real, seed = seed, min_count = 1, size = 50*j, 
                                         window = 5, alpha = 0.025, iter = 10, 
                                         batch_words = 10000, negative = 5, workers = 4)
    w2v_feature_array = averaged_word_vectorizer(corpus=cleaned_X_fake_or_real, model=word_model_X_fake_or_real,
                                             num_features=50*j)
    x0_train, x0_test, y0_train, y0_test = train_test_split(w2v_feature_array, Y_fake_or_real, test_size=test_size, random_state=seed)
    LR0_model = LogisticRegression().fit(x0_train, y0_train)
    y0_pred = LR0_model.predict(x0_test)
    
    acc=accuracy_score(y0_test, y0_pred)
    print("Word2Vec, size =", 50*j, ":",acc)
    

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Word2Vec, size = 50 : 0.8976566236250598
Word2Vec, size = 100 : 0.9014825442372071
Word2Vec, size = 150 : 0.9019607843137255
Word2Vec, size = 200 : 0.9029172644667623
Word2Vec, size = 250 : 0.9024390243902439
Word2Vec, size = 300 : 0.9048302247728359
Word2Vec, size = 350 : 0.9062649450023912


Lets fix the size as 300 as it the most optimal feature size according to previous work

### 2.3.2 Changing the window
Window parameter represent the distance between the current and predicted word, and we change it from 1 to 10.

In [41]:
# Testing the influence of window and use 50 as difference

for j in range(1,8):
    word_model_X_fake_or_real = Word2Vec(cleaned_X_fake_or_real, seed = seed, min_count = 1, size =300 , 
                                         window = j, alpha = 0.025, iter = 10, 
                                         batch_words = 10000, negative = 5, workers = 4)
    w2v_feature_array = averaged_word_vectorizer(corpus=cleaned_X_fake_or_real, model=word_model_X_fake_or_real,
                                             num_features=300)
    x0_train, x0_test, y0_train, y0_test = train_test_split(w2v_feature_array, Y_fake_or_real, test_size=test_size, random_state=seed)
    LR0_model = LogisticRegression().fit(x0_train, y0_train)
    y0_pred = LR0_model.predict(x0_test)
    
    acc=accuracy_score(y0_test, y0_pred)
    print("Word2Vec, window =", j, ":",acc)
    

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Word2Vec, window = 1 : 0.8971783835485414
Word2Vec, window = 2 : 0.9000478240076518
Word2Vec, window = 3 : 0.899091343854615
Word2Vec, window = 4 : 0.9029172644667623
Word2Vec, window = 5 : 0.9057867049258728
Word2Vec, window = 6 : 0.907221425155428
Word2Vec, window = 7 : 0.9062649450023912


The maximum accuracy of 0.907 is given by window size of 6 .Lets fix the window size as 6

## 2.3.3 Changing the alpha

The range of alpha is from 0.015 to 0.060, and the step is 0.005.

In [46]:
for j in range(1,8):
    word_model_X_fake_or_real = Word2Vec(cleaned_X_fake_or_real, seed = seed, min_count = 1, size =300 , 
                                          window = 6, alpha = 0.01+ 0.005*j, iter = 10, 
                                          batch_words = 10000, negative = 5, workers = 4)
    w2v_feature_array = averaged_word_vectorizer(corpus=cleaned_X_fake_or_real, model=word_model_X_fake_or_real,
                                             num_features=300)
    x0_train, x0_test, y0_train, y0_test = train_test_split(w2v_feature_array, Y_fake_or_real, test_size=test_size, random_state=seed)
    LR0_model = LogisticRegression().fit(x0_train, y0_train)
    y0_pred = LR0_model.predict(x0_test)
    
    acc=accuracy_score(y0_test, y0_pred)
    print("Word2Vec, alpha =", 0.01+ 0.005*j, ":",acc)


C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Word2Vec, alpha = 0.015 : 0.8995695839311334
Word2Vec, alpha = 0.02 : 0.9043519846963175
Word2Vec, alpha = 0.025 : 0.9014825442372071
Word2Vec, alpha = 0.03 : 0.907221425155428
Word2Vec, alpha = 0.035 : 0.9124820659971306
Word2Vec, alpha = 0.04 : 0.9115255858440937
Word2Vec, alpha = 0.045000000000000005 : 0.9158297465327594


The accuracy of 0.9115 is given by alpha of 0.4.Lets fix the alpha as 0.04


So the final parameters choosen for Word2Vec model are:
* size=300
* window size=6
* alpha=0.04

## 2.4 Choosing the Best Classifier for Word2Vec

Grid searches are typically used when you don't know (and often don't care too much about the meaning of) a set of optimal parameters to a given estimator or set of estimators. They are essentially a set of for loops that try out a series of parameters and construct a single model for each case (hence a grid). Scikits has a [grid search class](http://scikit-learn.org/stable/modules/grid_search.html#grid-search) that will automate an exhaustive or optimized search for one or more estimator parameters.



In [49]:
# The grid_search function

def grid_search(model, X, Y):
    grid_search = GridSearchCV(model, param, scoring="accuracy", n_jobs = 1)
    grid_result = grid_search.fit(X, Y)
    print("Best Accuracy:", grid_result.best_score_)
    print("Parameter set:", grid_result.best_params_)
    model.set_params(**grid_result.best_params_)
    model.fit(X, Y)
    y0_pred = model.predict(x0_test)
    accuracy_score(y0_test, y0_pred)

## 2.4.1 For Fake and Real News Dataset

In [50]:
# Use the best result from Word2Vec

word_model_X_fake_or_real = Word2Vec(cleaned_X_fake_or_real, seed = seed, min_count = 1, size =300 , 
                                     window = 6, alpha = 0.04, iter = 10, 
                                     batch_words = 10000, negative = 5, workers = 4)
w2v_feature_array = averaged_word_vectorizer(corpus=cleaned_X_fake_or_real, model=word_model_X_fake_or_real,
                                             num_features=300)
x0_train, x0_test, y0_train, y0_test = train_test_split(w2v_feature_array, Y_fake_or_real, test_size=test_size, random_state=seed)
    

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


## 2.4.1.1 Log Regression

In [51]:
from sklearn.model_selection import GridSearchCV
param = {
    'C': [1.0, 1.5, 2.0],
    'tol':[0.0001, 0.0002, 0.0003],
    'penalty': ['l1', 'l2'],
    'intercept_scaling': [1,2,3]
}

grid_search(LogisticRegression(random_state = 2),x0_train,y0_train)

Best Accuracy: 0.9205937794533459
Parameter set: {'C': 2.0, 'intercept_scaling': 3, 'penalty': 'l2', 'tol': 0.0001}


## Results
### Best Accuracy: 0.9205
### The best combination of parameters: 'C': 2.0, 'intercept_scaling': 3, 'penalty': 'l2', 'tol': 0.0001
 


## 2.4.1.2 Random Forest

In [52]:
from sklearn.ensemble import RandomForestClassifier
param = {
    'n_estimators': [10, 25, 50],
    'min_samples_split':[2, 3, 4],
    'verbose': [0, 3],
    'min_weight_fraction_leaf': [0.0, 0.1]
}

grid_search(RandomForestClassifier(random_state = 2),x0_train,y0_train)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10


...

building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


Best Accuracy: 0.9036286522148916
Parameter set: {'min_samples_split': 4, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'verbose': 0}


## Results
### Best Accuracy: 0.9036
### The best combination of parameters: {'min_samples_split': 4, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'verbose': 0}

## 2.4.1.3 XGBoost

In [53]:
from xgboost import XGBClassifier
XG0_model = XGBClassifier(max_depth=7, learning_rate=0.2, 
                        n_estimators=20, silent=True, 
                        objective='binary:logistic', nthread=-1, 
                        gamma=0, min_child_weight=1, max_delta_step=0, 
                        subsample=1, colsample_bytree=1, 
                        colsample_bylevel=1, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=1, 
                        base_score=0.5, seed=0, missing=None)
XG0_model = XG0_model.fit(x0_train, y0_train)
print("Word2Vec + XG:",XG0_model.score(x0_test, y0_test))
y0_pred = XG0_model.predict(x0_test)
print(accuracy_score(y0_test, y0_pred))

Word2Vec + XG: 0.9024390243902439
0.9024390243902439


C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [54]:
from xgboost import XGBClassifier
XG0_model = XGBClassifier(max_depth=6, learning_rate=0.2, 
                        n_estimators=20, silent=True, 
                        objective='binary:logistic', nthread=-1, 
                        gamma=0, min_child_weight=1, max_delta_step=0, 
                        subsample=1, colsample_bytree=1, 
                        colsample_bylevel=1, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=1, 
                        base_score=0.5, seed=0, missing=None)
XG0_model = XG0_model.fit(x0_train, y0_train)
print("Word2Vec + XG:",XG0_model.score(x0_test, y0_test))
y0_pred = XG0_model.predict(x0_test)
print(accuracy_score(y0_test, y0_pred))

Word2Vec + XG: 0.9086561453849833
0.9086561453849833


C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [55]:
from xgboost import XGBClassifier
XG0_model = XGBClassifier(max_depth=6, learning_rate=0.3, 
                        n_estimators=200, silent=True, 
                        objective='binary:logistic', nthread=-1, 
                        gamma=0, min_child_weight=2, max_delta_step=0, 
                        subsample=1, colsample_bytree=1, 
                        colsample_bylevel=1, reg_alpha=0, 
                        reg_lambda=3, scale_pos_weight=1, 
                        base_score=0.5, seed=0, missing=None)
XG0_model = XG0_model.fit(x0_train, y0_train)
print("Word2Vec + XG:",XG0_model.score(x0_test, y0_test))
y0_pred = XG0_model.predict(x0_test)
print(accuracy_score(y0_test, y0_pred))

Word2Vec + XG: 0.9143950263032042
0.9143950263032042


C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### XGBoost gives the best accuracy of 0.9143 with max_depth=6, learning_rate=0.3, n_estimators=200

## 2.4.2 For Liar Liar Dataset

In [44]:
# The grid_search function

def grid_search2(model, X, Y):
    grid_search = GridSearchCV(model, param, scoring="accuracy", n_jobs = 1)
    grid_result = grid_search.fit(X, Y)
    print("Best Accuracy:", grid_result.best_score_)
    print("Parameter set:", grid_result.best_params_)
    model.set_params(**grid_result.best_params_)
    model.fit(X, Y)
    y0_pred = model.predict(w2v_feature_array_test)
    accuracy_score(Y_test, y0_pred)

## 2.4.2.1 Logistic Regression

In [45]:
from sklearn.model_selection import GridSearchCV
param = {
    'C': [1.0, 1.5, 2.0],
    'tol':[0.0001, 0.0002, 0.0003],
    'penalty': ['l1', 'l2'],
    'intercept_scaling': [1,2,3]
}

grid_search2(LogisticRegression(random_state = 2),w2v_feature_array_train, Y_train)

Best Accuracy: 0.5460256760447965
Parameter set: {'C': 2.0, 'intercept_scaling': 1, 'penalty': 'l1', 'tol': 0.0003}


### Log Regression gives the best accuracy of 0.54 with 'C': 2.0, 'intercept_scaling': 1, 'penalty': 'l1', 'tol': 0.0003

## 2.4.2.2 Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
param = {
    'n_estimators': [10, 25, 50],
    'min_samples_split':[2, 3, 4],
    'verbose': [0, 3],
    'min_weight_fraction_leaf': [0.0, 0.1]
}

grid_search2(RandomForestClassifier(random_state = 2),w2v_feature_array_train,Y_train)

C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10
building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


...

building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50
building tree 1 of 50


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


Best Accuracy: 0.5566785031412183
Parameter set: {'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 50, 'verbose': 0}


### Random Forest gives highest accuracy of 0.55 with parameters 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 50, 'verbose': 0

## 2.4.2.3 XGBoost

In [48]:
from xgboost import XGBClassifier
XG0_model = XGBClassifier(max_depth=6, learning_rate=0.3, 
                        n_estimators=200, silent=True, 
                        objective='binary:logistic', nthread=-1, 
                        gamma=0, min_child_weight=2, max_delta_step=0, 
                        subsample=1, colsample_bytree=1, 
                        colsample_bylevel=1, reg_alpha=0, 
                        reg_lambda=3, scale_pos_weight=1, 
                        base_score=0.5, seed=0, missing=None)
XG0_model = XG0_model.fit(w2v_feature_array_train, Y_train)
print("Word2Vec + XG:",XG0_model.score(w2v_feature_array_test, Y_test))
y0_pred = XG0_model.predict(w2v_feature_array_test)
print(accburacy_score(Y_test, y0_pred))

Word2Vec + XG: 0.48358862144420134
0.48358862144420134


C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### With XGBoost, the accuracy is only 0.48

## Conclusion
***

* According to the results, the best classifiers for word2vec with "fake and real news dataset" are Logistic Regression and XGBoost with over 91% accuracy.
* However, XGBoost takes a lot more time than logistic regression and therefore, I wasnt able to apply GridCVSearch with XGBoost.
* Keeping the time and accuracy in perspective, Logistic Regression is better as compared to XGBoost with high accuracy and minimum time for the fake and real dataset.
* On the other hand, random forest gives the best results for the Liar Liar Dataset. 
* The performance of XGBoost with Word2Vec on Liar Liar dataset was surprisingly not vey high. 
* Hence, it can be concluded, the performance of different classifiers also depends on the context of the dataset and the features extracted thourgh Word2Vec since they have a direct link with the vocabulary developed.
* Best Results--> Word2Vec+Log Regression on fake and real dataset after tuning of the Word2Vec model and and tuning of hyperparameters of the classifiers

## References

* 1) https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
* 2) https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-10-neural-network-with-a6441269aa3c
* 3) https://arxiv.org/pdf/1301.3781.pdf
* 4) http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
* 5)https://towardsdatascience.com/understanding-feature-engineering-part-4-deep-learning-methods-for-text-data-96c44370bbfa?fbclid=IwAR0Eg7w8kFL71Pr6wnHZvdERXtd-J_WD-5Rg1ezqPMLIPWWwHzo-8bF-SfA